In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
from diffusers import DDIMScheduler
from pathlib import Path
from IPython import embed
import os

try:
    from local_sd_pipeline import LocalStableDiffusionPipeline
    from optim_utils import *
except ModuleNotFoundError:
    import os; os.chdir("..")
    from local_sd_pipeline import LocalStableDiffusionPipeline
    from optim_utils import *


/zhome/ca/2/153088/memorization/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Generate 100 "fake" validation prompts, save them in a new directory as .txt files

In [4]:
sentences = [
    "Patient is male and is 25 years old, enjoys playing basketball.",
    "Patient is female and is 38 years old, loves gardening on weekends.",
    "Patient is male and is 19 years old, enjoys watching movies.",
    "Patient is female and is 42 years old, likes to go hiking in the mountains.",
    "Patient is male and is 31 years old, enjoys cooking new recipes.",
    "Patient is female and is 55 years old, likes attending art exhibitions.",
    "Patient is male and is 28 years old, loves playing video games.",
    "Patient is female and is 60 years old, enjoys knitting in free time.",
    "Patient is male and is 22 years old, likes to go fishing.",
    "Patient is female and is 49 years old, enjoys reading mystery novels.",
    "Patient is male and is 35 years old, loves traveling to new countries.",
    "Patient is female and is 44 years old, enjoys going to the theater.",
    "Patient is male and is 27 years old, likes to go surfing.",
    "Patient is female and is 33 years old, enjoys photography as a hobby.",
    "Patient is male and is 29 years old, loves playing the guitar.",
    "Patient is female and is 50 years old, likes to go horseback riding.",
    "Patient is male and is 24 years old, enjoys playing tennis.",
    "Patient is female and is 36 years old, loves going to music concerts.",
    "Patient is male and is 21 years old, likes to go skateboarding.",
    "Patient is female and is 47 years old, enjoys painting landscapes.",
    "Patient is male and is 32 years old, loves watching football games.",
    "Patient is female and is 58 years old, likes to go birdwatching.",
    "Patient is male and is 26 years old, enjoys mountain biking.",
    "Patient is female and is 40 years old, loves going to the beach.",
    "Patient is male and is 30 years old, likes to go rock climbing.",
    "Patient is female and is 53 years old, enjoys gardening and growing vegetables.",
    "Patient is male and is 23 years old, loves playing chess.",
    "Patient is female and is 39 years old, likes to go camping in the wilderness.",
    "Patient is male and is 37 years old, enjoys playing golf.",
    "Patient is female and is 51 years old, loves going to dance classes.",
    "Patient is male and is 34 years old, likes to go kayaking.",
    "Patient is female and is 56 years old, enjoys writing poetry.",
    "Patient is male and is 20 years old, loves going to the gym.",
    "Patient is female and is 45 years old, likes to go scuba diving.",
    "Patient is male and is 41 years old, enjoys mountain climbing.",
    "Patient is female and is 54 years old, loves attending opera performances.",
    "Patient is male and is 46 years old, likes to go snowboarding in winter.",
    "Patient is female and is 48 years old, enjoys going to yoga classes.",
    "Patient is male and is 43 years old, loves playing basketball with friends.",
    "Patient is female and is 52 years old, likes to go horse riding in the countryside.",
    "Patient is male and is 59 years old, enjoys collecting rare coins.",
    "Patient is female and is 57 years old, loves going to wine tastings.",
    "Patient is male and is 61 years old, likes to go bungee jumping for excitement.",
    "Patient is female and is 62 years old, enjoys going to classical music concerts.",
    "Patient is male and is 63 years old, loves attending car shows.",
    "Patient is female and is 64 years old, likes to go ballroom dancing.",
    "Patient is male and is 65 years old, enjoys playing poker with friends.",
    "Patient is female and is 66 years old, loves going to book club meetings.",
    "Patient is male and is 67 years old, likes to go skydiving for adrenaline.",
    "Patient is female and is 68 years old, enjoys going to charity events.",
    "Patient is male and is 69 years old, loves playing billiards.",
    "Patient is female and is 70 years old, likes to go snorkeling in tropical waters.",
    "Patient is male and is 71 years old, enjoys going to antique auctions.",
    "Patient is female and is 72 years old, loves attending music festivals.",
    "Patient is male and is 73 years old, likes to go fly fishing.",
    "Patient is female and is 74 years old, enjoys going to pottery classes.",
    "Patient is male and is 75 years old, loves playing table tennis.",
    "Patient is female and is 76 years old, likes to go line dancing.",
    "Patient is male and is 77 years old, enjoys going to woodworking workshops.",
    "Patient is female and is 78 years old, loves attending theater plays.",
    "Patient is male and is 79 years old, likes to go hot air ballooning.",
    "Patient is female and is 80 years old, enjoys going to historical reenactments.",
    "Patient is male and is 81 years old, loves playing badminton.",
    "Patient is female and is 82 years old, likes to go sailing on weekends.",
    "Patient is male and is 83 years old, enjoys going to car races.",
    "Patient is female and is 84 years old, loves attending painting classes.",
    "Patient is male and is 85 years old, likes to go ice skating in winter.",
    "Patient is female and is 86 years old, enjoys going to tai chi sessions.",
    "Patient is male and is 87 years old, loves playing darts with friends.",
    "Patient is female and is 88 years old, likes to go archery.",
    "Patient is male and is 89 years old, enjoys going to film festivals.",
    "Patient is female and is 90 years old, loves attending cooking classes.",
    "Patient is male and is 91 years old, likes to go model airplane flying.",
    "Patient is female and is 92 years old, enjoys going to beekeeping workshops.",
    "Patient is male and is 93 years old, loves playing frisbee at the park.",
    "Patient is female and is 94 years old, likes to go rock hounding.",
    "Patient is male and is 95 years old, enjoys going to stargazing events.",
    "Patient is female and is 96 years old, loves attending poetry readings.",
    "Patient is male and is 97 years old, likes to go geocaching for adventure.",
    "Patient is female and is 98 years old, enjoys going to cheese tasting events.",
    "Patient is male and is 99 years old, loves playing shuffleboard.",
    "Patient is female and is 100 years old, likes to go beekeeping as a hobby."
]


In [5]:
output_dir = "/dtu/blackhole/14/207860/memo/training/martin/fake_validation_prompts/patient_sentences"

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through each sentence and save as a .txt file
for idx, sentence in enumerate(sentences, start=1):
    filename = f"patient_{idx}.txt"  # File name like patient_1.txt, patient_2.txt, ...
    filepath = os.path.join(output_dir, filename)
    with open(filepath, "w") as file:
        file.write(sentence)